In [77]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold
import matplotlib.pyplot as plt
from matplotlib import cm

In [78]:
#Load in the data
X = pd.read_csv("../Data/trainX.csv")
y = pd.read_csv("../Data/trainY.csv")
target_val = pd.read_csv("../Data/testX.csv")

['belongs_to_collection',
 'has_homepage',
 'originally_english',
 'topStudio',
 'usa_produced',
 'released_in_english',
 'has_tagline',
 'has_keywords',
 'topLeadActor',
 'topLeadDirector',
 'topLeadExecProd',
 'topLeadProducer',
 'topLeadComposer',
 'topLeadDirectorPhoto',
 'topLeadEditor',
 'log_budget_processed',
 'log_genre_rank',
 'log_num_genres',
 'log_overview_len',
 'log_numTopStudios',
 'log_num_studios',
 'log_num_production_countries',
 'log_day_of_week',
 'log_year',
 'log_month',
 'log_week_of_year',
 'log_season',
 'log_runtime_processed',
 'log_num_languages',
 'log_title_len',
 'log_num_cast',
 'log_numTopActors',
 'log_numTopDirectors',
 'log_numTopExecProd',
 'log_numTopProducers',
 'log_numTopComposers',
 'log_numTopDirectorsPhoto',
 'log_numTopEditors']

In [15]:
#X = X[['belongs_to_collection', 'has_homepage', 'originally_english', 'topStudio', 'usa_produced', 'released_in_english', 'has_tagline', 'has_keywords', 'topLeadActor', 'topLeadDirector', 'topLeadExecProd', 'topLeadProducer', 'topLeadComposer', 'topLeadDirectorPhoto', 'topLeadEditor', 'log_budget_processed', 'log_genre_rank', 'log_num_genres', 'log_overview_len', 'log_numTopStudios', 'log_num_studios', 'log_num_production_countries', 'log_day_of_week', 'log_year', 'log_month', 'log_week_of_year', 'log_season', 'log_runtime_processed', 'log_num_languages', 'log_title_len', 'log_num_cast', 'log_numTopActors', 'log_numTopDirectors', 'log_numTopExecProd', 'log_numTopProducers', 'log_numTopComposers', 'log_numTopDirectorsPhoto', 'log_numTopEditors']]

In [79]:
y = y['log_revenue']

In [80]:
# create a 75/25 stratified split of the data 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.20)

In [81]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (2399, 41) (2399,)
Test set: (600, 41) (600,)


In [19]:
y_test

1795    18.369387
1163    14.566924
1104    15.022289
1884    14.244405
1240    17.772392
          ...    
751     18.545058
2112    17.451265
2593    18.143367
1022    17.601012
150      2.944439
Name: log_revenue, Length: 600, dtype: float64

In [20]:
def algorithm_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, 
                       model, param_grid, cv=10, scoring_fit='neg_mean_squared_error',
                       do_probabilities = False):
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=-1, 
        scoring=scoring_fit,
        verbose=2
    )
    fitted_model = gs.fit(X_train_data, y_train_data)
    
    if do_probabilities:
        pred = fitted_model.predict_proba(X_test_data)
    else:
        pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred

In [21]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy

In [143]:
import math
def RMSLE(predict, target):
    total = 0 
    for k in range(len(predict)):
        LPred= np.log1p(predict[k]+1)
        LTarg = np.log1p(target[k] + 1)
        if not (math.isnan(LPred)) and  not (math.isnan(LTarg)): 
            total = total + ((LPred-LTarg) **2)
        
    total = total / len(predict)        
    return np.sqrt(total)

## Random Forest

In [135]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error
import seaborn as sns

In [146]:
rfr_base_model = RandomForestRegressor(random_state = 42)
rfr_base_model.fit(X_train, y_train.values.ravel())

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [147]:
y_pred = rfr_base_model.predict(X_test)

In [148]:
mse = mean_squared_error(y_test,y_pred)
rmse = np.sqrt(mse)
rmse

2.144078041153763

In [149]:
# Calculate the absolute errors
train_errors = abs(rfr_base_model.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(y_pred - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

Train Set Accuracy: 94.17 %
Test Set Accuracy: 85.21 %


In [150]:
RMSLE(np.expm1(y_pred), np.expm1(y_test.values))

2.1391284157001214

In [140]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [100, 125, 150],
    'max_features': ['auto'],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [5, 6, 8],
    'n_estimators': [300, 350]
}
# Create a based model
rfr = RandomForestRegressor()
# Instantiate the grid search model
rfr, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, rfr, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-rfr.best_score_))
print(rfr.best_params_)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  4.2min
[Parallel(n_jobs=-1)]: Done 270 out of 270 | elapsed:  7.8min finished


2.1683122503823666
{'bootstrap': True, 'max_depth': 100, 'max_features': 'auto', 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 350}


In [141]:
print('Base Model:')
base_accuracy = evaluate(rfr_base_model, X_test, y_test.values)
print()
print('Model after Tuning:')
rfr_best_model = rfr.best_estimator_
best_accuracy = evaluate(rfr_best_model, X_test, y_test.values)

print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

Base Model:
Average Error: 1.4115 degrees.
Accuracy = 85.21%.

Model after Tuning:
Average Error: 1.3802 degrees.
Accuracy = 84.91%.
Improvement of -0.35%.


In [142]:
feature_importances = rfr_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

Feature ranking:
1. log_budget_processed (0.449291)
2. log_num_male_crew (0.082359)
3. log_runtime_processed (0.046211)
4. log_num_cast (0.045132)
5. log_genre_rank (0.044908)
6. log_year (0.044292)
7. log_overview_len (0.040406)
8. log_week_of_year (0.030189)
9. topStudio (0.028379)
10. log_num_crew (0.024784)
11. log_title_len (0.022332)
12. log_num_studios (0.019592)
13. log_day_of_week (0.017817)
14. belongs_to_collection (0.010675)
15. log_season (0.008910)
16. log_num_genres (0.008874)
17. log_num_female_crew (0.008528)
18. log_num_production_countries (0.008493)
19. log_month (0.008137)
20. has_tagline (0.006799)
21. log_num_languages (0.006675)
22. has_keywords (0.005194)
23. log_numTopStudios (0.004784)
24. has_homepage (0.004635)
25. log_numTopDirectorsPhoto (0.003996)
26. usa_produced (0.003777)
27. log_numTopComposers (0.002536)
28. log_numTopActors (0.002475)
29. released_in_english (0.002412)
30. originally_english (0.002382)
31. log_numTopEditors (0.000902)
32. log_numTo

In [87]:
from sklearn.metrics import mean_squared_error,make_scorer
from hyperopt import tpe,hp,Trials
from hyperopt.fmin import fmin

In [88]:
seed=2
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=RandomForestRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train.values.ravel())
    pred=model.predict(X_test)
    score=mean_squared_error(y_test.values,pred)
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,250),
           'max_depth':hp.uniform('max_depth',5,100),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,10),
           'min_samples_split':hp.uniform('min_samples_split',2,8)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100,rstate=np.random.RandomState(seed))
    return best

trial=Trials()
best=optimize(trial)

100%|██████████| 100/100 [07:25<00:00,  4.46s/trial, best loss: 4.388124039960022]


In [89]:
print(best)

{'max_depth': 37.670296837858366, 'min_samples_leaf': 2.7543069442649797, 'min_samples_split': 7.556472297483721, 'n_estimators': 148.42037363056397}


In [133]:
rfr_opt=RandomForestRegressor(max_depth =38, min_samples_leaf= 3, min_samples_split = 8, n_estimators = 148)
rfr_opt.fit(X_train,y_train.values.ravel())
pred_rfr_opt=rfr_opt.predict(X_test)
score_rfr_opt=mean_squared_error(y_test.values,pred_rfr_opt)
print(np.sqrt(score_rfr_opt))

2.1252609573270678


In [134]:
# Calculate the absolute errors
train_errors = abs(rfr_opt.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(pred_rfr_opt - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

Train Set Accuracy: 90.74 %
Test Set Accuracy: 84.87 %


## Extra Trees

In [92]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from numpy import mean
from numpy import std

In [93]:
et_base_model = ExtraTreesRegressor()
et_base_model.fit(X_train, y_train.values.ravel());

In [94]:
y_pred = et_base_model.predict(X_test)

In [95]:

# Calculate the absolute errors
errors = abs(y_pred - y_test.values)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 1.47


In [96]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test.values)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 83.81 %.


In [97]:
et_model = ExtraTreesRegressor()
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

et_model, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, et_model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-et_model.best_score_))
print(et_model.best_params_)

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
base_model = ExtraTreesRegressor()
base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(base_model, X_test, y_test.values)

et_best_model = et_model.best_estimator_
best_accuracy = evaluate(et_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

Model Performance
Average Error: 2.8057 degrees.
Accuracy = 73.40%.
Model Performance
Average Error: 2.5961 degrees.
Accuracy = 74.66%.
Improvement of 1.71%.

In [ ]:
feature_importances = et_best_model.feature_importances_

In [ ]:
feature_importances = et_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))


In [104]:
seed=2
def objective(params):
    est=int(params['n_estimators'])
    md=int(params['max_depth'])
    msl=int(params['min_samples_leaf'])
    mss=int(params['min_samples_split'])
    model=ExtraTreesRegressor(n_estimators=est,max_depth=md,min_samples_leaf=msl,min_samples_split=mss)
    model.fit(X_train,y_train.values.ravel())
    pred=model.predict(X_test)
    score=mean_squared_error(y_test.values,pred)
    return score

def optimize(trial):
    params={'n_estimators':hp.uniform('n_estimators',100,250),
           'max_depth':hp.uniform('max_depth',5,100),
           'min_samples_leaf':hp.uniform('min_samples_leaf',1,10),
           'min_samples_split':hp.uniform('min_samples_split',2,8)}
    best=fmin(fn=objective,space=params,algo=tpe.suggest,trials=trial,max_evals=100,rstate=np.random.RandomState(seed))
    return best

trial=Trials()
best=optimize(trial)

100%|██████████| 100/100 [02:53<00:00,  1.74s/trial, best loss: 4.705339309316585]


In [105]:
print(best)

{'max_depth': 97.15489816691424, 'min_samples_leaf': 4.865393838386043, 'min_samples_split': 2.5167794377801482, 'n_estimators': 197.18586885757415}


In [108]:
et_opt=ExtraTreesRegressor(max_depth =97, min_samples_leaf= 5, min_samples_split = 2, n_estimators = 197)
et_opt.fit(X_train,y_train.values.ravel())
pred_et_opt=rfr_opt.predict(X_test)
score_et_opt=mean_squared_error(y_test.values,pred_rfr_opt)
print(np.sqrt(score_rfr_opt))

2.116443687657358


In [109]:
# Calculate the absolute errors
train_errors = abs(et_opt.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(pred_et_opt - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

Train Set Accuracy: 89.24 %
Test Set Accuracy: 85.06 %


In [145]:
RMSLE(np.expm1(pred_et_opt), np.expm1(y_test.values))

2.1109194626904055

## XGBoost

In [98]:
from numpy import loadtxt
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [99]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [100]:
xgb_base_model = xgb.XGBRegressor()
xgb_base_model.fit(X_train, y_train.values.ravel())

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [102]:
y_pred = xgb_base_model.predict(X_test)

In [103]:
# Calculate the absolute errors
train_errors = abs(xgb_base_model.predict(X_train) - y_train.values)
# Calculate mean absolute percentage error (MAPE)
train_mape = 100 * (train_errors / y_train.values)
# Calculate and display accuracy
train_accuracy = 100 - np.mean(train_mape)

# Calculate the absolute errors
test_errors = abs(y_pred - y_test.values)
# Calculate mean absolute percentage error (MAPE)
test_mape = 100 * (test_errors / y_test.values)
# Calculate and display accuracy
test_accuracy = 100 - np.mean(test_mape)
print('Train Set Accuracy:', round(train_accuracy, 2), '%')
print('Test Set Accuracy:', round(test_accuracy, 2), '%')

Train Set Accuracy: 97.56 %
Test Set Accuracy: 84.6 %


In [101]:
xgb_model = xgb.XGBRegressor()
param_grid = {
    'n_estimators': [100, 200, 400],
    'colsample_bytree': [1.0, 1.1, 1.5],
    'max_depth': [1, 5, 100],
    'reg_alpha': [1.2, 1.3, 1.4],
    'reg_lambda': [ 1.2, 1.3],
    'subsample': [0.8, 0.9, 1.0]
}

xgb_model, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, xgb_model, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-xgb_model.best_score_))
print(xgb_model.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
print('Base Model:')
base_accuracy = evaluate(xgb_base_model, X_test, y_test.values)
print()
print('Model after Tuning:')
xgb_best_model = xgb_model.best_estimator_
best_accuracy = evaluate(xgb_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))


In [ ]:
feature_importances = xgb_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

## LightGBM

In [ ]:
# lightgbm for regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot

# evaluate the model
lgbm_base_model = LGBMRegressor()
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
lgbm_base_model = LGBMRegressor()
lgbm_base_model.fit(X_train, y_train)
# make a single prediction
row = [[2.02220122, 0.31563495, 0.82797464, -0.30620401, 0.16003707, -1.44411381, 0.87616892, -0.50446586, 0.23009474, 0.76201118]]
yhat = lgbm_base_model.predict(X_test)
print('Prediction: %.3f' % yhat[0])

In [ ]:
mse = mean_squared_error(y_test,yhat)
rmse = np.sqrt(mse)
rmse

In [ ]:
# Calculate the absolute errors
errors = abs(yhat - y_test.values)
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test.values)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%')

In [ ]:
from sklearn.model_selection import GridSearchCV
# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [100, 150],
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8],
    'max_depth': [1, 2, 5],
    'reg_alpha': [1.0, 1.1, 1.2],
    'reg_lambda': [1.2, 1.3],
    'subsample': [0.3,0.35, 0.4]

}
# Create a based model
lgbm = LGBMRegressor()
lgbm, pred = algorithm_pipeline(X_train, X_test, y_train.values.ravel(), y_test.values, lgbm, 
                                 param_grid, cv=5)

# Root Mean Squared Error
print(np.sqrt(-lgbm.best_score_))
print(lgbm.best_params_)

In [ ]:
lgbm_base_model = LGBMRegressor()
lgbm_base_model.fit(X_train, y_train.values.ravel())
base_accuracy = evaluate(lgbm_base_model, X_test, y_test.values)

lgbm_best_model = lgbm.best_estimator_
best_accuracy = evaluate(lgbm_best_model, X_test, y_test.values)


print('Improvement of {:0.2f}%.'.format( 100 * (best_accuracy - base_accuracy) / base_accuracy))

In [ ]:
feature_importances = lgbm_best_model.feature_importances_
indices = np.argsort(feature_importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. %s (%f)" % (f + 1, X_train.columns[indices[f]], feature_importances[indices[f]]))

## Evaluation

In [128]:
results = pd.DataFrame(columns = ['id','revenue'])

In [111]:
preds=et_opt.predict(target_val)

In [127]:
for i in range(3001, len(target_val) + 3001):
    print(i)

3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148
3149
3150
3151
3152
3153
3154
3155
3156
3157
3158
3159
3160
3161
3162
3163
3164
3165
3166
3167
3168
3169
3170
3171
3172
3173
3174
3175
3176
3177
3178
3179
3180
3181
3182
3183
3184
3185
3186
3187
3188
3189
3190
3191
3192
3193
3194
3195
3196
3197
3198
3199
3200


5010
5011
5012
5013
5014
5015
5016
5017
5018
5019
5020
5021
5022
5023
5024
5025
5026
5027
5028
5029
5030
5031
5032
5033
5034
5035
5036
5037
5038
5039
5040
5041
5042
5043
5044
5045
5046
5047
5048
5049
5050
5051
5052
5053
5054
5055
5056
5057
5058
5059
5060
5061
5062
5063
5064
5065
5066
5067
5068
5069
5070
5071
5072
5073
5074
5075
5076
5077
5078
5079
5080
5081
5082
5083
5084
5085
5086
5087
5088
5089
5090
5091
5092
5093
5094
5095
5096
5097
5098
5099
5100
5101
5102
5103
5104
5105
5106
5107
5108
5109
5110
5111
5112
5113
5114
5115
5116
5117
5118
5119
5120
5121
5122
5123
5124
5125
5126
5127
5128
5129
5130
5131
5132
5133
5134
5135
5136
5137
5138
5139
5140
5141
5142
5143
5144
5145
5146
5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209


7211
7212
7213
7214
7215
7216
7217
7218
7219
7220
7221
7222
7223
7224
7225
7226
7227
7228
7229
7230
7231
7232
7233
7234
7235
7236
7237
7238
7239
7240
7241
7242
7243
7244
7245
7246
7247
7248
7249
7250
7251
7252
7253
7254
7255
7256
7257
7258
7259
7260
7261
7262
7263
7264
7265
7266
7267
7268
7269
7270
7271
7272
7273
7274
7275
7276
7277
7278
7279
7280
7281
7282
7283
7284
7285
7286
7287
7288
7289
7290
7291
7292
7293
7294
7295
7296
7297
7298
7299
7300
7301
7302
7303
7304
7305
7306
7307
7308
7309
7310
7311
7312
7313
7314
7315
7316
7317
7318
7319
7320
7321
7322
7323
7324
7325
7326
7327
7328
7329
7330
7331
7332
7333
7334
7335
7336
7337
7338
7339
7340
7341
7342
7343
7344
7345
7346
7347
7348
7349
7350
7351
7352
7353
7354
7355
7356
7357
7358
7359
7360
7361
7362
7363
7364
7365
7366
7367
7368
7369
7370
7371
7372
7373
7374
7375
7376
7377
7378
7379
7380
7381
7382
7383
7384
7385
7386
7387
7388
7389
7390
7391
7392
7393
7394
7395
7396
7397
7398


In [129]:
results['id'] = range(3001, len(target_val) + 3001)
results['revenue'] = np.expm1(preds)
results

,id,revenue
0,3001,6.033100e+06
1,3002,1.077699e+06
2,3003,1.178528e+07
3,3004,9.919555e+06
4,3005,1.005190e+06
...,...,...
4393,7394,2.928679e+07
4394,7395,7.464092e+07
4395,7396,2.930045e+07
4396,7397,1.990279e+06


In [130]:
results.to_csv("ET_baseline2-85p.csv",index=False)